In [1]:
!pip install -q grad-cam==1.4.3
!pip install -q wandb
!pip install -q segmentation_models_pytorch
!pip install -q torchattacks
!pip install -q monai
!pip install -q torchsummary

# from kaggle_datasets import KaggleDatasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.27.2 requires requests>=2.31, but you have requests 2.25.1 which is incompatible.


In [3]:
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)
import torchaudio
print(torchaudio.__version__)

2.3.1+cpu
0.18.1+cpu
2.3.1+cpu


In [1]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, random_split
import albumentations as A

import segmentation_models_pytorch as smp
# import torchsummary

import pandas as pd
import numpy as np
import random, shutil, time, os

import sklearn
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import albumentations as A

from glob import glob
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import roc_auc_score
# from skimage import color
from IPython import display as ipd

import scipy
import pdb
import gc

import torchattacks
import monai

from pytorch_grad_cam import GradCAM


from torch.cuda import amp

import warnings
warnings.filterwarnings('ignore')

print('done')

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


done


In [2]:
import torch

# 检查是否有可用的CUDA设备
if torch.cuda.is_available():
    # 如果有，打印CUDA设备数量和名称
    print(f"CUDA is available! Number of devices: {torch.cuda.device_count()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is available! Number of devices: 1
Device name: NVIDIA GeForce RTX 4090


In [3]:
CFG = {
    'lr':3e-4,
    'shape':(224, 224),

}
TRAIN = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
def seed_everything(seed=44):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def clear_cache():
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
DATA_DIR = 'E:/Chrome Download/uw-madison-gi-tract-image-segmentation'

# Open the training dataframe and display the initial dataframe
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)

In [6]:
DATA_DIR = 'E:/Chrome Download/uw-madison-gi-tract-image-segmentation'

# Open the training dataframe and display the initial dataframe
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)

all_train_images = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive=True)


def get_filepath_from_partial_identifier(_ident, file_list):
    return [x for x in file_list if _ident in x][0]

def df_preprocessing(df, globbed_file_list, is_test=False):
    """ The preprocessing steps applied to get column information """
    # 1. Get Case-ID as a column (str and int)
    df["case_id_str"] = df["id"].apply(lambda x: x.split("_", 2)[0])
    df["case_id"] = df["id"].apply(lambda x: int(x.split("_", 2)[0].replace("case", "")))

    # 2. Get Day as a column
    df["day_num_str"] = df["id"].apply(lambda x: x.split("_", 2)[1])
    df["day_num"] = df["id"].apply(lambda x: int(x.split("_", 2)[1].replace("day", "")))

    # 3. Get Slice Identifier as a column
    df["slice_id"] = df["id"].apply(lambda x: x.split("_", 2)[2])

    # 4. Get full file paths for the representative scans
    df["_partial_ident"] = (globbed_file_list[0].rsplit("/", 4)[0]+"/"+ "Chrome Download/uw-madison-gi-tract-image-segmentation/train/"+
                           df["case_id_str"]+"/"+ # .../case###/
                           df["case_id_str"]+"_"+df["day_num_str"]+ # .../case###_day##/
                           "/scans/"+df["slice_id"]) # .../slice_####
    _tmp_merge_df = pd.DataFrame({"_partial_ident":[x.rsplit("_",4)[0] for x in globbed_file_list], "f_path":globbed_file_list})
    df = df.merge(_tmp_merge_df, on="_partial_ident").drop(columns=["_partial_ident"])

    # 5. Get slice dimensions from filepath (int in pixels)
    df["slice_h"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))
    df["slice_w"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))

    # 6. Pixel spacing from filepath (float in mm)
    df["px_spacing_h"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[3]))
    df["px_spacing_w"] = df["f_path"].apply(lambda x: float(x[:-4].rsplit("_",4)[4]))

    if not is_test:
        # 7. Merge 3 Rows Into A Single Row (As This/Segmentation-RLE Is The Only Unique Information Across Those Rows)
        l_bowel_df = df[df["class"]=="large_bowel"][["id", "segmentation"]].rename(columns={"segmentation":"lb_seg_rle"})
        s_bowel_df = df[df["class"]=="small_bowel"][["id", "segmentation"]].rename(columns={"segmentation":"sb_seg_rle"})
        stomach_df = df[df["class"]=="stomach"][["id", "segmentation"]].rename(columns={"segmentation":"st_seg_rle"})
        df = df.merge(l_bowel_df, on="id", how="left")
        df = df.merge(s_bowel_df, on="id", how="left")
        df = df.merge(stomach_df, on="id", how="left")
        df = df.drop_duplicates(subset=["id",]).reset_index(drop=True)
        df["lb_seg_flag"] = df["lb_seg_rle"].apply(lambda x: not pd.isna(x))
        df["sb_seg_flag"] = df["sb_seg_rle"].apply(lambda x: not pd.isna(x))
        df["st_seg_flag"] = df["st_seg_rle"].apply(lambda x: not pd.isna(x))
        df["n_segs"] = df["lb_seg_flag"].astype(int)+df["sb_seg_flag"].astype(int)+df["st_seg_flag"].astype(int)

    # 8. Reorder columns to the a new ordering (drops class and segmentation as no longer necessary)
    new_col_order = ["id", "f_path", "n_segs",
                     "lb_seg_rle", "lb_seg_flag",
                     "sb_seg_rle", "sb_seg_flag",
                     "st_seg_rle", "st_seg_flag",
                     "slice_h", "slice_w", "px_spacing_h",
                     "px_spacing_w", "case_id_str", "case_id",
                     "day_num_str", "day_num", "slice_id",]
    if is_test: new_col_order.insert(1, "class")
    new_col_order = [_c for _c in new_col_order if _c in df.columns]
    df = df[new_col_order]
    return df






# df_preprocessing(train_df, all_train_images)
# all_test_images = glob(os.path.join(TEST_DIR, "**", "*.png"), recursive=True)


df = pd.read_csv("E:/Chrome Download/uw-madison-gi-tract-image-segmentation/train.csv")
df['segmentation'] = df.segmentation.fillna('')

df['rle_len'] = df.segmentation.map(len) # length of each rle mask

df2 = df.groupby(['id'])['segmentation'].agg(list).to_frame().reset_index() # rle list of each id
df2["f_path"] = all_train_images
df2 = df2.merge(df.groupby(['id'])['rle_len'].agg(sum).to_frame().reset_index()) # total length of all rles of each id

df = df.drop(columns=['segmentation', 'class', 'rle_len'])
df = df.groupby(['id']).head(1).reset_index(drop=True)
df = df.merge(df2, on=['id'])
df['empty'] = (df.rle_len==0) # empty masks

# 1. Get Case-ID as a column (str and int)
df["case_id_str"] = df["id"].apply(lambda x: x.split("_", 2)[0])
df["case_id"] = df["id"].apply(lambda x: int(x.split("_", 2)[0].replace("case", "")))

# 2. Get Day as a column
df["day_num_str"] = df["id"].apply(lambda x: x.split("_", 2)[1])
df["day_num"] = df["id"].apply(lambda x: int(x.split("_", 2)[1].replace("day", "")))

# 3. Get Slice Identifier as a column
df["slice_id"] = df["id"].apply(lambda x: x.split("_", 2)[2])

# 4. Get full file paths for the representative scans
# df["_partial_ident"] = (all_train_images[0].rsplit("/", 4)[0]+"/"+ "Chrome Download/uw-madison-gi-tract-image-segmentation/train/"+
#                        df["case_id_str"]+"/"+ # .../case###/
#                        df["case_id_str"]+"_"+df["day_num_str"]+ # .../case###_day##/
#                        "/scans/"+all_train_images[0].rsplit("\\", 4)[-1]) # .../slice_####



# 5. Get slice dimensions from filepath (int in pixels)
df["slice_h"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[1]))
df["slice_w"] = df["f_path"].apply(lambda x: int(x[:-4].rsplit("_",4)[2]))

df.rename(columns={
    'f_path':'path',
    'slice_h':'img_height',
    'slice_w':'img_width',
    'case_id':'case',
    'day_num':'day'
}, inplace=True)

df['slice'] = df['slice_id'].apply(lambda a : int(a.split('_')[1]))

df.drop(columns=['slice_id', 'case_id_str', 'day_num_str'], inplace=True)
if TRAIN:
    fault1 = 'case7_day0'
    fault2 = 'case81_day30'
    df = df[~df['id'].str.contains(fault1) & ~df['id'].str.contains(fault2)].reset_index(drop=True)

df['lb'] = df['segmentation'].map(lambda a: a[0] if a[0] != '' else '')
df['sb'] = df['segmentation'].map(lambda a: a[1] if a[1] != '' else '')
df['st'] = df['segmentation'].map(lambda a: a[2] if a[2] != '' else '')

df['classes'] = df['segmentation'].map(lambda a: [(a[0] != '') + 0, (a[1] != '') + 0, (a[2] != '') + 0 ])
np.random.seed(80)
df = df.sample(frac=1).reset_index(drop=True)
print('done')


done


In [7]:
# https://www.kaggle.com/paulorzp/rle-functions-run-length-encode-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_decode(mask_rle, wid, hei):
    shape = (wid, hei)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


def img_read(path):
    img = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
    return img


class Dataset2D(torch.utils.data.Dataset):
    def __init__(self, df_sub, train=True):
        self.train = train

        self.paths = np.array(df_sub['path'])
        self.rles = np.array(df_sub['segmentation'])
        self.classes = np.array(df_sub['classes'])
        self.wid = np.array(df_sub['img_width'])
        self.hei = np.array(df_sub['img_height'])


    def __len__(self):
        return len(self.paths)

    def transform(self, img, mask):
        trans = A.Compose([
#             A.ToFloat(max_value=65535.0), # essential because albu requires 32 bits!!! ONLY THIS can force it work with 16 bits!!

            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.5),


            A.ShiftScaleRotate(
                scale_limit=0.12,  # 0
                shift_limit=0.02,  # 0.05
                rotate_limit=15,
                border_mode=cv2.BORDER_CONSTANT,
                value=(1,1,1),
                always_apply=True,
                p=1,
            ),

            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),

            # A.OneOf([
            #         A.ElasticTransform(
            #             alpha=1,
            #             sigma=25,
            #             always_apply=True,
            #         ),
            #         A.GridDistortion(
            #             always_apply=True,
            #         ),
            #         A.OpticalDistortion(
            #             distort_limit=0.05,
            #             shift_limit=0.05,
            #             always_apply=True,
            #         ),
            #     ], p=1
            # ),
        ])
        return trans(image=img, mask=mask)


    def data_prep_aug(self, img, mask, classes):
        shape = CFG['shape']
        img = (cv2.resize(img, shape, interpolation=cv2.INTER_AREA) / img.max()).astype('float32')
        mask = cv2.resize(mask, shape, interpolation=cv2.INTER_AREA).astype('float32')

        if self.train:
            trans = self.transform(img, mask)
            img = trans['image'].reshape((1, shape[0], shape[1]))
            mask = trans['mask']

#         # normalize
#         img = (img - img.min()) / (img.max() - img.min())
        blank_img = np.zeros((shape[0], shape[1], 3))
        blank_img[:, :, 0] = img
        blank_img[:, :, 1] = img
        blank_img[:, :, 2] = img
        img = blank_img.transpose(2,1,0)

#         plt.imshow(img.reshape(256, 256, 3))
#         plt.pause(1)

#         mask_final = np.zeros((len(classes), shape[0], shape[1]))
#         for i in range(len(classes)):
#             mask_final[i, :, :] = mask[:,:,i]
        mask = mask.transpose(2,1,0)

        return torch.tensor(img, dtype=torch.float16, device=device), torch.tensor(mask, dtype=torch.float16, device=device)

    def __getitem__(self, idx):
        img = img_read(self.paths[idx])

        blank_mask = np.zeros((self.wid[idx],  self.hei[idx], 3))
        blank_mask[:, :, 0] = rle_decode(self.rles[idx][0], self.wid[idx], self.hei[idx])
        blank_mask[:, :, 1] = rle_decode(self.rles[idx][1], self.wid[idx], self.hei[idx])
        blank_mask[:, :, 2] = rle_decode(self.rles[idx][2], self.wid[idx], self.hei[idx])

        # data preprocessing and augmentation
        img, masks = self.data_prep_aug(img, blank_mask, self.classes[idx])

        return img, masks

In [8]:
def imshow(img, return_only=False, pause=False, show_axis=True):
    if isinstance(img, np.ndarray):
        if len(img.shape) == 4:
            img = img[0]
        if img.shape[0] == 3:
            img = img.transpose(2,1,0)

    if isinstance(img, torch.Tensor):
        if len(img.shape) == 4:
            img = img.cpu().detach()[0].numpy().transpose(2,1,0)
        elif len(img.shape) == 3:
            if img.shape[0] == 3:
                img = img.cpu().detach().numpy().transpose(2,1,0)
        elif len(img.shape) == 2:
            img = img.cpu().detach().numpy()

    if return_only:
        return img
    else:
#         plt.figure(figsize=(5,5))
        plt.subplots()
        plt.imshow(img)
        if pause:
            plt.pause(1)
        if not show_axis:
            plt.axis('off')

In [9]:
idx = 62

def read(idx):
    img = cv2.imread(df['path'][idx], cv2.IMREAD_ANYDEPTH)
    shape = CFG['shape']
    img = (cv2.resize(img, shape, interpolation=cv2.INTER_AREA) / img.max()).astype('float32')

    blank_mask = np.zeros((df.img_width[idx],  df.img_height[idx], 3))
    blank_mask[:, :, 0] = rle_decode(df.segmentation[idx][0], df.img_width[idx], df.img_height[idx])
    blank_mask[:, :, 1] = rle_decode(df.segmentation[idx][1], df.img_width[idx], df.img_height[idx])
    blank_mask[:, :, 2] = rle_decode(df.segmentation[idx][2], df.img_width[idx], df.img_height[idx])
    mask = blank_mask
    mask = cv2.resize(mask, shape, interpolation=cv2.INTER_AREA).astype('float32').transpose(2,1,0).reshape(1, 3, shape[0], shape[1])

    blank_img = np.zeros((shape[0], shape[1], 3))
    blank_img[:, :, 0] = img
    blank_img[:, :, 1] = img
    blank_img[:, :, 2] = img
    img = blank_img.transpose(2,1,0).reshape((1, 3, shape[0], shape[1]))

    return img, mask


def predict(idx, model, to_numpy=True, log=True):
    img, mask = read(idx)

    model.eval()

    mask = torch.tensor(mask, device=device,dtype=torch.float)
    img = torch.tensor(img, device=device,dtype=torch.float)
    pred = torch.sigmoid(model(img))

    pred[pred < 0.5] = 0
    pred[pred > 0.5] = 1
    pred[pred == 0.5] = 1

    if log:
        dl = monai.losses.DiceLoss()(mask, pred)
        print((1-dl.cpu().detach().numpy()))


    if to_numpy:
        pred = pred.cpu().detach().numpy()
        img = img.cpu().detach().numpy()
        mask = mask.cpu().detach().numpy()

    return img, mask, pred

In [10]:
USE_WANDB = False
if USE_WANDB:
    import wandb
    from wandb.keras import WandbCallback
    secret_value = 'UR WANDB KEY!'
    wandb.login(key=secret_value)

    # wandb.init(project='unet_tract_tumor')

In [11]:
# VGG13
activation = None
model_vgg = smp.Unet(
    encoder_weights=None,
    encoder_name='vgg13',
    decoder_use_batchnorm=True,
    activation=activation,
    in_channels=3,
    classes=3,
)
model_vgg = model_vgg.to(device)
model_vgg.load_state_dict(torch.load("E:/adv_attk/archive (3)/unet_vgg13_12.15epochs_lr3e4.pt"))
print(1)

# UNet ResNeXt101
activation = None
model_res = smp.Unet(
    encoder_weights=None,
    encoder_name='resnext101_32x8d',
    decoder_use_batchnorm=True,
    activation=activation,
    in_channels=3,
    classes=3,
)
model_res = model_res.to(device)
model_res.load_state_dict(torch.load("E:/adv_attk/archive (1)/unet_resnext101_focaldice_12.15epochs_lr3e4.pt"))
print(2)

# UNet EFFB7
activation = None
model_eff = smp.Unet(
    encoder_weights=None,
    encoder_name='efficientnet-b7',
    decoder_use_batchnorm=True,
    activation=activation,
    in_channels=3,
    classes=3,
)
model_eff = model_eff.to(device)
model_eff.load_state_dict(torch.load("E:/adv_attk/archive/unet_effb7_NEW11.15.pt"))
print(3)

# OLD UNet2p EFFB7
activation = None
model_2p = smp.UnetPlusPlus(
    encoder_weights=None,
    encoder_name='efficientnet-b7',
    decoder_use_batchnorm=True,
    activation=activation,
    in_channels=3,
    classes=3,
)
model_2p = model_2p.to(device)
model_2p.load_state_dict(torch.load("E:/adv_attk/archive (2)/unet2p_effb7_focaldice_13.15epochs_lr3e4.pt"))
print('finished')

1
2
3
finished


# Transformer Model

In [12]:
import wandb
run = wandb.init()
artifact = run.use_artifact('veb-101/UM_medical_segmentation/model-jsr2fn8v:v0', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: wilsonyjk. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-jsr2fn8v:v0, 977.89MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.1


In [12]:
%pip install -qqqU wandb transformers lightning albumentations torchmetrics torchinfo
%pip install -qqq requests gradio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import os
import zipfile
import platform
import warnings
from glob import glob
from dataclasses import dataclass
 
# To filter UserWarning.
warnings.filterwarnings("ignore", category=UserWarning)
 
import cv2
import requests
import numpy as np
# from tqdm import tqdm
import matplotlib.pyplot as plt
 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
 
 
# For data augmentation and preprocessing.
import albumentations as A
from albumentations.pytorch import ToTensorV2
 
# Imports required SegFormer classes
from transformers import SegformerForSemanticSegmentation
 
# Importing lighting along with a built-in callback it provides.
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
 
# Importing torchmetrics modular and functional implementations.
from torchmetrics import MeanMetric
from torchmetrics.classification import MulticlassF1Score
 
# To print model summary.
from torchinfo import summary
 
# Sets the internal precision of float32 matrix multiplications.
torch.set_float32_matmul_precision('high')
 
# To enable determinism.
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"
 
# To render the matplotlib figure in the notebook.
%matplotlib inline

In [22]:
gkf = GroupKFold(n_splits=10)

# model = monai.networks.nets.UNet(
#     spatial_dims=2,
#     in_channels=3,
#     out_channels=3,

#     channels=(32, 64, 128, 256, 512),
# #     strides=(2, 2, 2, 2),
#     num_res_units=2,
# )

epochs = 15
train_bs = 16

num_epoch_2_skip = 0

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'])
criterion = lambda y_pred, y_true : monai.losses.DiceFocalLoss(sigmoid=(activation == None))(y_pred, y_true)
# criterion = lambda y_pred, y_true : 1 - dice_coef(y_pred, y_true)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=int(30000/train_bs*epochs)+50, eta_min=5e-6,)



n_accumulate = 1

for fold, (train_ind, val_ind) in tqdm(enumerate(gkf.split(df, df['empty'], groups=df['case'])), desc='Train '):

    if USE_WANDB:
        wandb.init(name = f"PostRUNS", project="unet_tract_tumor_v6", entity="kagglers",
                    config = CFG, save_code = False, group = "UNet V3_2",
                    notes = "with some da, lr=2e-3")
        wandb.watch(model, log='all')


    best_val_dice = -1
    for epoch in range(1, epochs+1):
        clear_cache()
        seed_everything(44)
        val_dice = 0
        if epoch <= num_epoch_2_skip:
            continue

        print(f'Epoch {epoch}/{epochs}', end='')

        model.train()
        scaler = torch.cuda.amp.GradScaler(init_scale=65536.0)

        dataset_size = 0
        running_loss = 0.0

        train_ds = Dataset2D(df.iloc[train_ind], train=True)
        train_ds_loader = torch.utils.data.DataLoader(train_ds, batch_size=train_bs)

        val_ds = Dataset2D(df.iloc[val_ind], train=False)
        val_ds_loader = torch.utils.data.DataLoader(val_ds, batch_size=8)


        # ========================================
        # TRAINING
        # ========================================

        pbar = tqdm(enumerate(train_ds_loader), total=len(train_ds_loader), desc='Train ')

        for step, (images, masks) in pbar:
            batch_size = images.size(0)

            with amp.autocast(enabled=True):
                y_pred = model(images)
                loss   = criterion(y_pred, masks)

            (scaler.scale(loss)/n_accumulate).backward()

            if (step + 1) % n_accumulate == 0:
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
                scaler.step(optimizer)
                scaler.update()

                optimizer.zero_grad()

                if scheduler is not None:
                    scheduler.step()

            running_loss += (loss.item() * batch_size)
            current_loss = (loss.item())
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size
            current_lr = optimizer.param_groups[0]['lr']


            if np.isnan(epoch_loss):
                print('NAN LOSS')
                break

            if USE_WANDB:
                wandb.log({
                    'train_current_loss':current_loss,
                    'lr':current_lr,
                })

            pbar.set_postfix(
                train_loss=f'{epoch_loss:0.4f}',
                current_loss=f'{current_loss:0.5f}',
                lr=f'{current_lr:0.6f}',
            )
        if USE_WANDB:
            wandb.log({
                'train_epoch_loss':epoch_loss,
            })
        torch.save(model.state_dict(), 'ptunetbaseline_v2.pt')


        clear_cache()


        # ========================================
        # Validation
        # ========================================

        model.eval()

        dataset_size = 0
        running_loss = 0.0

        pbar = tqdm(enumerate(val_ds_loader), total=len(val_ds_loader), desc='Valid ')

        for step, (images, masks) in pbar:
            images = images.to(dtype=torch.float, device=device)
            masks = masks.to(dtype=torch.float, device=device)

            batch_size = images.size(0)

            y_pred  = model(images)
            loss    = criterion(y_pred, masks)

            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            y_pred = nn.Sigmoid()(y_pred)

            y_pred[y_pred < 0.5] = 0
            y_pred[y_pred > 0.5] = 1

            dl = monai.losses.DiceLoss()(masks, y_pred)
            val_dice += (1-dl.cpu().detach().numpy())
#             val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()

            current_lr = optimizer.param_groups[0]['lr']
            if USE_WANDB:
                wandb.log({
                    'running_valid_loss':epoch_loss,
                })
            pbar.set_postfix(
                valid_loss=f'{epoch_loss:0.4f}',
                dice_acc=f'{val_dice}'
            )

        if USE_WANDB:
            wandb.log({
                'val_dice':val_dice,
                'valid_loss':epoch_loss,
            })

        if val_dice > best_val_dice:
            best_val_dice = val_dice
            print('saving...')
            torch.save(model.state_dict(), f'ptunetbaseline_v2_{epoch}.pt')

    if USE_WANDB:
        wandb.finish()
    break

Train : 0it [00:00, ?it/s]

Epoch 1/15

Train :   0%|          | 0/2152 [00:00<?, ?it/s]

AttributeError: 'ImageClassifierOutput' object has no attribute 'last_hidden_state'

In [ ]:
poisoned_val = 0
poisoned_val_counts = 0
model.eval()

for a in tqdm(val_ind):
#     img, mask, pred = predict(a, model=model_vgg, to_numpy=False, log=False)
    img, mask = read(a)
    if mask.max() != 0:
        mask = torch.tensor(mask, device=device, dtype=torch.float)

        pred = torch.sigmoid(model(torch.tensor(img, device=device, dtype=torch.float)))
        pred[pred > 0.5] = 1
        pred[pred == 0.5] = 1
        pred[pred < 0.5] = 0


        dl = monai.losses.DiceLoss()(mask, pred)
        poisoned_val += (1-dl.cpu().detach().numpy())
        poisoned_val_counts += 1

# #         -- --- --- , loss=torch.nn.BCEWithLogitsLoss()
#         poisoned_img = FGSM_attack(model=model, img=img, mask=mask, eps=0.009)
#         poisoned_img[poisoned_img > 1] = 1
#
#         pred = torch.sigmoid(model(poisoned_img))
#         mask = torch.tensor(mask, device=device, dtype=torch.float, )
#
#         pred[pred > 0.5] = 1
#         pred[pred == 0.5] = 1
#         pred[pred < 0.5] = 0
#
#         dl = monai.losses.DiceLoss()(mask, pred)
#         poisoned_val += (1-dl.cpu().detach().numpy())
#         poisoned_val_counts += 1
# #     else:
# #         pred = torch.sigmoid(model(torch.tensor(img, device=device, dtype=torch.float)))
# #         pred[pred > 0.5] = 1
# #         pred[pred == 0.5] = 1
# #         pred[pred < 0.5] = 0
# #         if torch.sum(pred) != 0:
# #             print(a)
# # #             imshow(pred)


In [ ]:
poisoned_val / poisoned_val_counts